In [257]:
import requests as rq
from bs4 import BeautifulSoup as soup
import pandas as pd
import os
import pubchempy as pcp
import re

In [135]:
df_keys = ['cid','name']
df = pd.DataFrame(columns = df_keys)
df

,cid,name


In [ ]:
df_dic = {}

In [160]:
def get_cid(cid):
    return cid

In [162]:
a = get_cid(1983)
a

1983

In [164]:
def get_name(response):
    return response['Record']['RecordTitle']

In [315]:
def get_atc_code(response):
    pat = r"\"[A-Z]\d{2}[A-Z]{2}\d{2}\""
    atc_code_found = re.search(pat, response.text)
    if atc_code_found:
        return atc_code_found.group(0).strip('"')

In [328]:
cid = 1980
url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON'
response = rq.get(url)
get_atc_code(response)


In [330]:
cids = list(range(1980,1990))
df_keys = ['cid', 'name', 'atc_code']
df_atc = pd.DataFrame(columns = df_keys)
df_no_atc = pd.DataFrame(columns = df_keys)
df_dic = {}

for cid in cids:
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON'
    response_json = rq.get(url).json()
    response_text = rq.get(url)
    df_dic['cid'] = get_cid(cid)
    df_dic['name'] = get_name(response_json)
    df_dic['atc_code'] = get_atc_code(response_text)
    if get_atc_code(response_text):
        df_atc.loc[len(df_atc)] = df_dic
    else:
        df_no_atc.loc[len(df_no_atc)] = df_dic


display(df_atc)
display(df_no_atc)

,cid,name,atc_code
0,1981,Acemetacin,M01AB11
1,1983,Acetaminophen,N02BE01
2,1985,Acetarsol,G01AB01
3,1986,Acetazolamide,S01EC01
4,1989,Acetohexamide,A10BB31


,cid,name,atc_code
0,1980,"3,6-Diacetoxy-3,3a,6,6a-tetrahydrofuro[3,2-b]f...",None
1,1982,Acephate,None
2,1984,Acetaminosalol,None
3,1987,"Acetic acid n-[2-chloro-5-[6-ethyl-2,4-diamino...",None
4,1988,Acetochlor,None


In [256]:
cids = list(range(1980, 1990))
for cid in cids:
    print(cid)
    
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON'
    response = rq.get(url).json()
    for head in response['Record']['Section']:
        if head['TOCHeading'] == 'Pharmacology and Biochemistry':
            for key in head['Section']:
                #print(key)
                if key['TOCHeading'] == 'ATC Code':
                    for prop in key['Information']:
                        if 'StringWithMarkup' in prop['Value'].keys() :
                            print(prop['Value']['StringWithMarkup'][0]['String'])
                       


1980
1981
M01AB11
M - Musculo-skeletal system
1982
1983
N02BE01
N - Nervous system
1984
1985
G - Genito urinary system and sex hormones
P - Antiparasitic products, insecticides and repellents
A - Alimentary tract and metabolism
1986
S01EC01
S - Sensory organs
1987
1988
1989
A - Alimentary tract and metabolism


In [313]:
cids = list(range(1980, 1990))
for cid in cids:
    print(cid)
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/json'
    response = rq.get(url)
    pat = r"\"[A-Z]\d{2}[A-Z]{2}\d{2}\""
    atc_code_found = re.search(pat, response.text)
    if atc_code_found:
        print(atc_code_found.group(0).strip('"'))
    

1980
1981
M01AB11
1982
1983
N02BE01
1984
1985
G01AB01
1986
S01EC01
1987
1988
1989
A10BB31


In [310]:
cid = 1983
url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/json'
response = rq.get(url)
pat = r"\"[A-Z]\d{2}[A-Z]{2}\d{2}\""
atc_code =  re.search(pat, response.text)
print(atc_code.group(0).strip('"'))


N02BE01


In [250]:
for head in response['Record']['Section']:
    if head['TOCHeading'] == 'Pharmacology and Biochemistry':
        for key in head['Section']:
            #print(key)
            if key['TOCHeading'] == 'ATC Code':
                for prop in key['Information']:
                    print(prop)
                    

{'ReferenceNumber': 172, 'Reference': ["S76 | LUXPHARMA | Pharmaceuticals Marketed in Luxembourg | Pharmaceuticals marketed in Luxembourg, as published by d'Gesondheetskeess (CNS, la caisse nationale de sante, www.cns.lu), mapped by name to structures using CompTox by R. Singh et al. (in prep.). List downloaded from https://cns.public.lu/en/legislations/textes-coordonnes/liste-med-comm.html. Dataset DOI:10.5281/zenodo.4587355"], 'URL': 'https://www.whocc.no/atc_ddd_index/?code=N02BE01', 'Value': {'StringWithMarkup': [{'String': 'N02BE01'}]}}
{'ReferenceNumber': 225, 'Value': {'StringWithMarkup': [{'String': 'N - Nervous system', 'Markup': [{'Start': 0, 'Length': 1, 'URL': 'https://www.whocc.no/atc_ddd_index/?code=N'}]}, {'String': 'N02 - Analgesics', 'Markup': [{'Start': 0, 'Length': 3, 'URL': 'https://www.whocc.no/atc_ddd_index/?code=N02'}]}, {'String': 'N02B - Other analgesics and antipyretics', 'Markup': [{'Start': 0, 'Length': 4, 'URL': 'https://www.whocc.no/atc_ddd_index/?code=N02

In [183]:
drugbank_df = pd.read_csv('..\Drugbank database\drugbank_all_full_database.xml\molecule_dataframe.csv')
drugbank_df.columns


Index(['Unnamed: 0', '@type', '@created', '@updated', 'drugbank-id', 'name',
       'cas-number', 'unii', 'average-mass', 'monoisotopic-mass', 'groups',
       'mixtures', 'manufacturers', 'experimental-properties', 'snp-effects',
       'snp-adverse-drug-reactions', 'targets', 'enzymes', 'carriers',
       'transporters', 'logP', 'logS', 'Water Solubility', 'IUPAC Name',
       'Traditional IUPAC Name', 'Molecular Weight', 'Monoisotopic Weight',
       'SMILES', 'Molecular Formula', 'InChI', 'InChIKey',
       'Polar Surface Area (PSA)', 'Refractivity', 'Polarizability',
       'Rotatable Bond Count', 'H Bond Acceptor Count', 'H Bond Donor Count',
       'pKa (strongest acidic)', 'Physiological Charge', 'Number of Rings',
       'Bioavailability', 'Rule of Five', 'Ghose Filter', 'MDDR-Like Rule',
       'atc_code', 'atc_code_0', 'atc_code_0_$', 'atc_code_1', 'atc_code_1_$',
       'atc_code_2', 'atc_code_2_$', 'atc_code_3', 'atc_code_3_$'],
      dtype='object')

In [145]:
comp = pcp.get_compounds(1983)

In [146]:
comp[0]

Compound(1983)

'{\n  "Record": {\n    "RecordType": "CID",\n    "RecordNumber": 1983,\n    "RecordTitle": "Acetaminophen",\n    "Section": [\n      {\n        "TOCHeading": "Structures",\n        "Description": "Structure depictions of this compound, including computationally generated two-dimensional (2D) and three-dimensional (3D) structures, as well as experimentally determined 3D single-crystal structures.",\n        "Section": [\n          {\n            "TOCHeading": "2D Structure",\n            "Description": "A two-dimensional (2D) structure representation of the compound.  Because this structure is processed through chemical structure standardization (Hähnke et al., J. Cheminform. 2018, 10, 36), it is not necessarily the same as the structures provided by individual data contributors.  ",\n            "URL": "https://doi.org/10.1186/s13321-018-0293-8",\n            "DisplayControls": {\n              "MoveToTop": true\n            },\n            "Information": [\n              {\n          

N02BE01
